In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv

# Load .env from repo root (next to this notebook)
dotenv_path = Path("..") / ".env"
loaded = load_dotenv(dotenv_path)
print(f".env found: {dotenv_path.resolve().exists()}  loaded: {loaded}")

def env_str(key, default=None, required=False):
    val = os.getenv(key, default)
    if required and (val is None or str(val).strip() == ""):
        raise ValueError(f"Missing required env var: {key}")
    return val

def env_int(key, default=None, required=False):
    v = env_str(key, default, required)
    return int(v) if v is not None else None

def env_float(key, default=None, required=False):
    v = env_str(key, default, required)
    return float(v) if v is not None else None

.env found: True  loaded: True


In [2]:
from datetime import datetime
import json, hashlib
import pandas as pd
from google.maps import routeoptimization_v1 as ro

# Pull config from env
PROJECT       = env_str("GOOGLE_CLOUD_PROJECT", required=True)
PARENT        = f"projects/{PROJECT}"

# If you provided a service account JSON, ADC will use it automatically
gac = env_str("GOOGLE_APPLICATION_CREDENTIALS", default=None)
if gac:
    p = Path(gac)
    print(f"GOOGLE_APPLICATION_CREDENTIALS set -> exists: {p.exists()}  path: {p}")

DEPOT_LAT=10.2509284
DEPOT_LNG=105.978708
FLEET_SIZE    = env_int("FLEET_SIZE", 10)

GLOBAL_START  = env_str("GLOBAL_START", "2025-10-28T01:00:00Z")
GLOBAL_END    = env_str("GLOBAL_END",   "2025-10-28T15:00:00Z")

DELIVERIES_CSV = Path(env_str("DELIVERIES_CSV", "data/deliveries.csv"))

print("Config loaded from env:")
print("  PARENT:", PARENT)
print("  DEPOT:", (DEPOT_LAT, DEPOT_LNG))
print("  FLEET_SIZE:", FLEET_SIZE)
print("  TIME WINDOW:", GLOBAL_START, "→", GLOBAL_END)
print("  FILE:", DELIVERIES_CSV.resolve())

Config loaded from env:
  PARENT: projects/hidden-terrain-476501-e6
  DEPOT: (10.2509284, 105.978708)
  FLEET_SIZE: 10
  TIME WINDOW: 2025-10-28T01:00:00Z → 2025-10-28T15:00:00Z
  FILE: /Users/huynhtai/Desktop/4. projects/VHoang/project/data/deliveries.csv


In [3]:
def parse_latlng_str(val: str):
    parts = [p.strip() for p in str(val).split(",")]
    if len(parts) < 2:
        raise ValueError(f"Bad coordinates string: {val}")
    return float(parts[0]), float(parts[1])

def latlng(lat, lng):
    return {"latitude": float(lat), "longitude": float(lng)}

def gmaps_url(origin, waypoints, destination):
    def fmt(ll): return f"{ll[0]},{ll[1]}"
    parts = [
        "api=1",
        f"origin={fmt(origin)}",
        f"destination={fmt(destination)}",
    ]
    if waypoints:
        parts.append("waypoints=" + "|".join(fmt(w) for w in waypoints))
    return "https://www.google.com/maps/dir/?" + "&".join(parts)

print("Helpers ready: parse_latlng_str, latlng, gmaps_url")

Helpers ready: parse_latlng_str, latlng, gmaps_url


In [4]:
assert DELIVERIES_CSV.exists(), f"Missing file: {DELIVERIES_CSV}"
df = pd.read_csv(DELIVERIES_CSV)
df.columns = [c.strip().lower() for c in df.columns]

required_cols = {"label", "coordinates"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"deliveries.csv missing columns: {missing}")

print(f"Loaded deliveries: {len(df)} rows")
print(df.head(10))

# Build label -> (lat,lng)
label_to_ll = {}
bad_rows = []
for idx, row in df.iterrows():
    try:
        label = str(row["label"]).strip()
        dlat, dlng = parse_latlng_str(row["coordinates"])
        label_to_ll[label] = (dlat, dlng)
    except Exception as e:
        bad_rows.append((idx, str(e)))

if bad_rows:
    print("Warning: bad rows encountered:")
    for idx, msg in bad_rows[:10]:
        print("  row", idx, "->", msg)
else:
    print("All delivery coordinates parsed successfully.")

print("First 5 label→coords:", list(label_to_ll.items())[:5])


Loaded deliveries: 97 rows
                                               label             coordinates
0  19 Nguyen Tat Thanh, Phu Thuy, Phan Thiet City...  10.9287267,108.1103265
1  9 Nguyễn Đình Chiểu Street, Ward 1, Bến Tre Ci...  10.3493245,107.0759918
2  84 Đường 30/4, Ward 2, Bạc Liêu City, Bạc Liêu...  10.5253636,107.1854734
3  2 Cong Xa Paris, Ben Nghe Ward, District 1, Ho...  10.7800898,106.6995646
4  16 Phan Chu Trinh Street, Ward 1, Trà Vinh Cit...  10.9320161,106.8654049
5  9A Đường 22 Tháng 12, Thuận Giao Ward, Thuận A...   10.924871,107.2413524
6  7 Nguyễn Văn Cừ Street, Mỹ Thới Ward, Long Xuy...   11.001337,106.6671784
7  45 Nguyễn Trãi Street, Ward 4, Vị Thanh City, ...   10.9463042,106.680124
8  186 30/4 Street, Ward 3, Tay Ninh City, Tay Ni...  11.3351554,106.1098854
9  No. 1, National Highway 1, Ward 2, Tan An City...   10.5651354,106.420883
All delivery coordinates parsed successfully.
First 5 label→coords: [('19 Nguyen Tat Thanh, Phu Thuy, Phan Thiet City, Binh Th

In [5]:
shipments = []
for label, (dlat, dlng) in label_to_ll.items():
    shipments.append({
        "label": label,
        "deliveries": [
            { "arrival_location": latlng(dlat, dlng) }
        ]
    })

print(f"Shipments built: {len(shipments)}")
print("Preview of first 3 shipments:")
print(json.dumps(shipments[:3], indent=2))


Shipments built: 96
Preview of first 3 shipments:
[
  {
    "label": "19 Nguyen Tat Thanh, Phu Thuy, Phan Thiet City, Binh Thuan, Vietnam",
    "deliveries": [
      {
        "arrival_location": {
          "latitude": 10.9287267,
          "longitude": 108.1103265
        }
      }
    ]
  },
  {
    "label": "9 Nguy\u1ec5n \u0110\u00ecnh Chi\u1ec3u Street, Ward 1, B\u1ebfn Tre City, B\u1ebfn Tre Province, Vietnam",
    "deliveries": [
      {
        "arrival_location": {
          "latitude": 10.3493245,
          "longitude": 107.0759918
        }
      }
    ]
  },
  {
    "label": "84 \u0110\u01b0\u1eddng 30/4, Ward 2, B\u1ea1c Li\u00eau City, B\u1ea1c Li\u00eau Province, Vietnam",
    "deliveries": [
      {
        "arrival_location": {
          "latitude": 10.743141234670162,
          "longitude": 106.63003946176038
        }
      }
    ]
  }
]


In [6]:
vehicles = []
for i in range(1, FLEET_SIZE + 1):
    vehicles.append({
        "label": f"V{i}",
        "start_location": latlng(DEPOT_LAT, DEPOT_LNG),
        "end_location":   latlng(DEPOT_LAT, DEPOT_LNG)
    })

print(f"Vehicles built: {len(vehicles)}")
print("Vehicle labels:", [v["label"] for v in vehicles])
print("First vehicle preview:")
print(json.dumps(vehicles[0], indent=2))

Vehicles built: 10
Vehicle labels: ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10']
First vehicle preview:
{
  "label": "V1",
  "start_location": {
    "latitude": 10.2509284,
    "longitude": 105.978708
  },
  "end_location": {
    "latitude": 10.2509284,
    "longitude": 105.978708
  }
}


In [7]:
model = {
    "shipments": shipments,
    "vehicles": vehicles,
    "global_start_time": GLOBAL_START,
    "global_end_time": GLOBAL_END
}

print("Model summary:")
print("  shipments:", len(model["shipments"]))
print("  vehicles:", len(model["vehicles"]))
print("  window:", model["global_start_time"], "→", model["global_end_time"])
print("Model preview (first shipment, first vehicle):")
print(json.dumps({"shipment": model["shipments"][0], "vehicle": model["vehicles"][0]}, indent=2))


Model summary:
  shipments: 96
  vehicles: 10
  window: 2025-10-28T01:00:00Z → 2025-10-28T15:00:00Z
Model preview (first shipment, first vehicle):
{
  "shipment": {
    "label": "19 Nguyen Tat Thanh, Phu Thuy, Phan Thiet City, Binh Thuan, Vietnam",
    "deliveries": [
      {
        "arrival_location": {
          "latitude": 10.9287267,
          "longitude": 108.1103265
        }
      }
    ]
  },
  "vehicle": {
    "label": "V1",
    "start_location": {
      "latitude": 10.2509284,
      "longitude": 105.978708
    },
    "end_location": {
      "latitude": 10.2509284,
      "longitude": 105.978708
    }
  }
}


In [8]:
print(json.dumps(model, indent=2))

{
  "shipments": [
    {
      "label": "19 Nguyen Tat Thanh, Phu Thuy, Phan Thiet City, Binh Thuan, Vietnam",
      "deliveries": [
        {
          "arrival_location": {
            "latitude": 10.9287267,
            "longitude": 108.1103265
          }
        }
      ]
    },
    {
      "label": "9 Nguy\u1ec5n \u0110\u00ecnh Chi\u1ec3u Street, Ward 1, B\u1ebfn Tre City, B\u1ebfn Tre Province, Vietnam",
      "deliveries": [
        {
          "arrival_location": {
            "latitude": 10.3493245,
            "longitude": 107.0759918
          }
        }
      ]
    },
    {
      "label": "84 \u0110\u01b0\u1eddng 30/4, Ward 2, B\u1ea1c Li\u00eau City, B\u1ea1c Li\u00eau Province, Vietnam",
      "deliveries": [
        {
          "arrival_location": {
            "latitude": 10.743141234670162,
            "longitude": 106.63003946176038
          }
        }
      ]
    },
    {
      "label": "2 Cong Xa Paris, Ben Nghe Ward, District 1, Ho Chi Minh City, Vietnam",
    

In [9]:
client = ro.RouteOptimizationClient()
req = ro.OptimizeToursRequest(
    parent=f"projects/{PROJECT}",
    model=model,
    consider_road_traffic=False,
    populate_polylines=False
)
print("Calling OptimizeTours...")
resp = client.optimize_tours(request=req)
print("Response received.")
print("Number of routes returned:", len(resp.routes))


/Users/huynhtai/Desktop/4. projects/VHoang/env/lib/python3.13/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Calling OptimizeTours...
Response received.
Number of routes returned: 10


In [10]:
depot = (DEPOT_LAT, DEPOT_LNG)

summary_rows = []
for route in resp.routes:
    vlabel = route.vehicle_label
    ordered_labels = [v.shipment_label for v in route.visits if v.shipment_label]
    waypoints = [label_to_ll[lbl] for lbl in ordered_labels if lbl in label_to_ll]
    url = gmaps_url(origin=depot, waypoints=waypoints, destination=depot)

    print(f"\nVehicle {vlabel}:")
    print("  Stops:", " -> ".join(ordered_labels) if ordered_labels else "(no stops)")
    print("  Maps:", url)

    summary_rows.append({
        "vehicle": vlabel,
        "stops_count": len(ordered_labels),
        "stops_order": " -> ".join(ordered_labels),
        "maps_url": url
    })

summary_df = pd.DataFrame(summary_rows).sort_values(["vehicle"]).reset_index(drop=True)
summary_df


Vehicle V1:
  Stops: 27 Đường Hùng Vương, Thị Trấn Tân Minh, Hàm Tân District, Bình Thuận Province, Vietnam -> 17 Lê Quý Đôn Street, Ward 2, Bà Rịa City, Bà Rịa–Vũng Tàu Province, Vietnam -> 9 Phan Đình Phùng Street, Thị Trấn Long Điền, Long Điền District, Bà Rịa–Vũng Tàu Province, Vietnam -> 21 Trần Hưng Đạo Street, Ward 3, Sa Đéc City, Đồng Tháp Province, Vietnam -> 5 Võ Thị Sáu Street, Phước Hiệp Ward, Bà Rịa City, Bà Rịa–Vũng Tàu Province, Vietnam -> 12 Trương Định Street, Ward 8, Vũng Tàu City, Bà Rịa–Vũng Tàu Province, Vietnam -> 11 Nguyễn Văn Linh Street, An Hòa Ward, Rạch Giá City, Kiên Giang Province, Vietnam -> 2A Cách Mạng Tháng 8 Street, Ward 1, Sóc Trăng City, Sóc Trăng Province, Vietnam -> 52 Phan Văn Hớn Street, Hòa Phú Ward, Thuận An City, Bình Dương Province, Vietnam -> 6 Quốc lộ 1A, Bình Trị Đông B Ward, Bình Tân District, Ho Chi Minh City, Vietnam -> 5 Lý Thường Kiệt Street, Ward 7, Cà Mau City, Cà Mau Province, Vietnam -> 123 Quốc lộ 1A, Bình Trị Đông B Ward, Bình 

,vehicle,stops_count,stops_order,maps_url
0,V1,25,"27 Đường Hùng Vương, Thị Trấn Tân Minh, Hàm Tâ...",https://www.google.com/maps/dir/?api=1&origin=...
1,V10,0,,https://www.google.com/maps/dir/?api=1&origin=...
2,V2,10,"Hung Vuong I Residential Area, Phu Thuy Ward, ...",https://www.google.com/maps/dir/?api=1&origin=...
3,V3,6,"Lot B, An Phú - An Khánh New Urban Area, An Ph...",https://www.google.com/maps/dir/?api=1&origin=...
4,V4,11,"15 Trần Hưng Đạo Street, Ward 3, Vĩnh Long Cit...",https://www.google.com/maps/dir/?api=1&origin=...
5,V5,15,"73 Nguyễn Trãi Street, Ward 1, Gò Công Town, T...",https://www.google.com/maps/dir/?api=1&origin=...
6,V6,0,,https://www.google.com/maps/dir/?api=1&origin=...
7,V7,12,"Nguyen Ai Quoc Street, Quang Vinh Ward, Bien H...",https://www.google.com/maps/dir/?api=1&origin=...
8,V8,0,,https://www.google.com/maps/dir/?api=1&origin=...
9,V9,17,"1231 Quốc lộ 1A, Bình Trị Đông B Ward, Bình Tâ...",https://www.google.com/maps/dir/?api=1&origin=...


In [ ]:
out_path = Path(env_str("ROUTES_CSV", "data/routes.csv"))
summary_df.to_csv(out_path, index=False)
print("Wrote:", out_path.resolve())

Wrote: /Users/huynhtai/Desktop/4. projects/VHoang/project/data/routes.csv


: 